# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries


import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
X =  df.message.values

Y =  df.iloc[:,5:]
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
   
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
  
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:

pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
pipe.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fead658a730>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

pipe.fit(X_train, Y_train)




Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
def eval_metrics(array1, array2, col_names):
    metrics = []
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        print(col_names[i])
        print(classification_report(array1[:,i],array2[:,i]))

In [13]:
Y_test_pred = pipe.predict(X_test)
col_names = list(Y.columns.values)
eval_metrics(np.array(Y_test), Y_test_pred, col_names)


request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5472
          1       0.83      0.42      0.56      1125

avg / total       0.88      0.89      0.87      6597

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6561
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6597

aid_related
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3865
          1       0.76      0.53      0.63      2732

avg / total       0.74      0.74      0.73      6597

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6097
          1       0.57      0.08      0.14       500

avg / total       0.90      0.93      0.90      6597

medical_products
             precision    recall  f1-score   support

          0       0.95      1.

In [14]:
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)
eval_metrics(np.array(Y_train), Y_train_pred, col_names)


request
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16401
          1       1.00      0.93      0.97      3388

avg / total       0.99      0.99      0.99     19789

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19704
          1       1.00      0.79      0.88        85

avg / total       1.00      1.00      1.00     19789

aid_related
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     11567
          1       1.00      0.97      0.98      8222

avg / total       0.99      0.99      0.99     19789

medical_help
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     18190
          1       1.00      0.87      0.93      1599

avg / total       0.99      0.99      0.99     19789

medical_products
             precision    recall  f1-score   support

          0       0.99      1.

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}


cv_fit = GridSearchCV(pipe, param_grid = parameters,verbose = 1 , n_jobs =-1)

# Find best parameters
np.random.seed(74)
cv_fit.fit(X_train, Y_train)


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 11.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

##### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
y_predtest = cv_fit.predict(X_test)
y_predtrain = cv_fit.predict(X_train)

eval_ypred_test = eval_metrics(np.array(Y_test), y_predtest, col_names)
eval_ypred_train = eval_metrics(np.array(Y_test), y_predtrain, col_names)

print(eval_ypred_test)
print(eval_ypred_train )



request
             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5472
          1       1.00      0.00      0.00      1125

avg / total       0.86      0.83      0.75      6597

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6561
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6597

aid_related
             precision    recall  f1-score   support

          0       0.61      0.99      0.76      3865
          1       0.88      0.12      0.21      2732

avg / total       0.72      0.63      0.53      6597

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6097
          1       0.00      0.00      0.00       500

avg / total       0.85      0.92      0.89      6597

medical_products
             precision    recall  f1-score   support

          0       0.95      1.

ValueError: Found input variables with inconsistent numbers of samples: [6597, 19789]

##### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
#using decision tree classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

cv_fit2 = GridSearchCV(pipeline2, param_grid = parameters, verbose = 3)
cv_fit2.fit(X_train,Y_train)
tuned_pred_tes2 = cv_fit2.predict(X_test)
eval_cv_tes2 = eval_metrics(np.array(Y_test), tuned_pred_tes2,col_names)


print(eval_cv_tes2)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.3662270729119297, total=  28.6s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.3s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.36628259551243175, total=  28.4s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.35809581564584597, total=  28.1s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.36440806427163863, total=  27.8s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.3693147362037599, total=  28.0s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.3585506367495452, total=  27.2s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, score=0.3700166742458693, total=  26.9s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, score=0

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 44.4min finished


request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      5472
          1       0.73      0.52      0.61      1125

avg / total       0.88      0.89      0.88      6597

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6561
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      6597

aid_related
             precision    recall  f1-score   support

          0       0.72      0.86      0.79      3865
          1       0.73      0.53      0.62      2732

avg / total       0.73      0.73      0.72      6597

medical_help
             precision    recall  f1-score   support

          0       0.94      0.98      0.96      6097
          1       0.50      0.20      0.28       500

avg / total       0.90      0.92      0.91      6597

medical_products
             precision    recall  f1-score   support

          0       0.96      0.

TypeError: estimator should be an estimator implementing 'fit' method, [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f18be70f950>, vocabulary=None)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('logistic', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.1,
          verbose=0, warm_start=False))] was passed

### 9. Export your model as a pickle file

In [19]:


m = pickle.dumps('classifier.pkl') 



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.